<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/RK_AMM_Simulation_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class RiskAssessmentAgent:
    """
    Simulates an AI agent that assesses borrower risk.
    In a real system, this would be a complex ML model.
    Here, it generates risk profiles for the simulation.
    """
    def __init__(self):
        # We can make this more complex later, e.g., link to reputation
        pass

    def assess_borrower(self, borrower_id, reputation_score=0.5):
        """
        Generates a default probability (p_d) and collateral level (C).
        A higher reputation score should lead to a lower p_d.
        """
        # Simple heuristic: base_p_d is random, but reputation improves it
        base_p_d = np.random.uniform(0.01, 0.40) # Base risk

        # Reputation impact: 0.5 is neutral. 1.0 is max reputation, 0.0 is min.
        # This is a simple model we can refine.
        p_d = base_p_d * (1 - (reputation_score * 0.5))

        # Assume collateral is determined by risk profile
        collateral_level = max(0, 0.8 - p_d * 2) # Higher risk, more collateral

        # Ensure p_d is within a realistic-but-risky range
        p_d = np.clip(p_d, 0.01, 0.99)
        collateral_level = np.clip(collateral_level, 0.0, 0.8)

        return p_d, collateral_level

class PricingAgent:
    """
    Uses the Reverse Kelly AMM logic from Esteva et al. (2023)
    to calculate the required premium for a loan.
    """
    def __init__(self):
        pass

    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):
        """
        Calculates the premium (P) based on rkAMM logic.

        Args:
        p_d (float): Probability of Default (from RiskAssessmentAgent)
        collateral_level (float): Collateral percentage (e.g., 0.6 for 60%)
        loan_amount (float): The principal amount of the loan (normalized to 1.0)

        Returns:
        float: The premium required to insure the loan.
        float: The Loss Given Default (L)
        """

        # Loss Given Default (L) is the portion of the loan *not* covered
        # by collateral.
        L = loan_amount * (1.0 - collateral_level)

        if L == 0:
            # Fully collateralized, no risk, no premium
            return 0.0, L

        if p_d == 0:
            # No default risk, no premium
            return 0.0, L

        if p_d >= 1.0:
            # Certain default, loan is impossible
            return np.inf, L

        # The core rkAMM formula (Esteva et al., 2023)
        # P = (L * p_d) / (1 - p_d)
        # This is the "fair" premium to cover the expected loss in a
        # risk-neutral sense, adjusted by the Kelly criterion.
        premium = (L * p_d) / (1.0 - p_d)

        return premium, L

class BlockchainEnforcementAgent:
    """
    Simulates the execution of the loan on the blockchain.
    In our simulation, this agent tracks outcomes.
    """
    def __init__(self):
        pass

    def execute_loan(self, p_d):
        """
        Simulates if a loan defaults or is repaid.

        Returns:
        bool: True if repaid, False if defaulted.
        """
        # A simple Bernoulli trial
        if np.random.rand() < p_d:
            return False # Default
        else:
            return True # Repaid

class Simulation:
    """
    Runs the full simulation for the MAS-rkAMM system.
    """
    def __init__(self, num_loans):
        self.num_loans = num_loans
        self.risk_agent = RiskAssessmentAgent()
        self.pricing_agent = PricingAgent()
        self.blockchain_agent = BlockchainEnforcementAgent()
        self.results = []

    def run(self):
        """
        Run the simulation for `num_loans`
        """
        lp_pool_balance = 0.0

        for i in range(self.num_loans):
            # 1. Assess risk
            # For simplicity, we'll use a random reputation score
            reputation = np.random.uniform(0.1, 0.9)
            p_d, collateral_level = self.risk_agent.assess_borrower(i, reputation)

            # 2. Price the loan
            premium, loss_given_default = self.pricing_agent.calculate_premium(p_d, collateral_level)

            # 3. LP pool collects the premium
            lp_pool_balance += premium

            # 4. Execute loan and check outcome
            repaid = self.blockchain_agent.execute_loan(p_d)

            if repaid:
                loan_outcome = "Repaid"
                profit_loss = premium # LP pool keeps the premium
            else:
                loan_outcome = "Default"
                # LP pool pays out the loss, which is covered by the premiums
                # The net effect is (premium - loss)
                profit_loss = premium - loss_given_default
                lp_pool_balance -= loss_given_default

            self.results.append({
                "loan_id": i,
                "p_d": p_d,
                "collateral_level": collateral_level,
                "loss_given_default": loss_given_default,
                "premium": premium,
                "outcome": loan_outcome,
                "profit_loss": profit_loss,
                "lp_pool_balance": lp_pool_balance
            })

        return pd.DataFrame(self.results)

if __name__ == "__main__":
    # --- Run the Simulation ---
    N_LOANS = 10000
    print(f"Running MAS-rkAMM Simulation with {N_LOANS} loans...")

    sim = Simulation(num_loans=N_LOANS)
    df_results = sim.run()

    print("Simulation Complete.")
    print("\n--- Results Snapshot ---")
    print(df_results.head())

    print("\n--- Key Metrics ---")
    total_premiums = df_results['premium'].sum()
    total_losses = df_results[df_results['outcome'] == 'Default']['loss_given_default'].sum()
    net_profit = total_premiums - total_losses
    avg_lp_yield_per_loan = net_profit / N_LOANS

    print(f"Total Premiums Collected: {total_premiums:.4f}")
    print(f"Total Losses Paid (from defaults): {total_losses:.4f}")
    print(f"LP Pool Net Profit: {net_profit:.4f}")
    print(f"Average LP Yield per Loan: {avg_lp_yield_per_loan:.6f}")
    print(f"Total Defaults: {len(df_results[df_results['outcome'] == 'Default'])}")

    # --- Plotting ---
    plt.figure(figsize=(14, 6))

    # 1. Plot LP Pool Balance
    plt.subplot(1, 2, 1)
    plt.plot(df_results['loan_id'], df_results['lp_pool_balance'], label='LP Pool Balance', color='blue')
    plt.title('LP Pool Balance Over Time')
    plt.xlabel('Loan ID')
    plt.ylabel('Cumulative Profit/Loss')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()

    # 2. Plot Premiums vs. Default Probability
    plt.subplot(1, 2, 2)
    plt.scatter(df_results['p_d'], df_results['premium'], alpha=0.3, label='Calculated Premium')
    plt.title('Premium vs. Probability of Default (p_d)')
    plt.xlabel('Probability of Default (p_d)')
    plt.ylabel('Premium')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()

    plt.tight_layout()
    plt.show()

    print("\nThis simulation shows the rkAMM model successfully prices risk,")
    print("leading to a positive expected return for the LP pool, even")
    print("with a high number of defaults.")